In [1]:
import pandas as pd
from usp_stream_datasets import *

pd.set_option('mode.use_inf_as_na', True)

/tmp/ipykernel_50091/4071251299.py:4: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('mode.use_inf_as_na', True)


In [9]:
dataset = "Abrupt (bal.)"
change_points = insects_datasets[dataset]["change_point"]
df = load_insect_dataset(insects_datasets[dataset]["filename"])

first_abrupt_change_point = change_points[0]

df_train = df.iloc[:first_abrupt_change_point].copy()
df_test = df.iloc[first_abrupt_change_point:].copy()
del df

In [14]:
y = df_train.pop("class")
X = df_train

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_hat = le.fit_transform(y)

# y_hat
# le.inverse_transform(y_hat)

## TODO: Data preparation

Given the nature of SVC, maybe is a good thing to properly scale some attributes.

In [22]:
# numeric_features = ["age", "fare"]

# numeric_transformer = Pipeline(
#     steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
# )

# categorical_features = ["embarked", "sex", "pclass"]

# categorical_transformer = Pipeline(
#     steps=[
#         ("encoder", OneHotEncoder(handle_unknown="ignore")),
#         ("selector", SelectPercentile(chi2, percentile=50)),
#     ]
# )

# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", numeric_transformer, numeric_features),
#         ("cat", categorical_transformer, categorical_features),
#     ]
# )

# clf = Pipeline(
#     steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
# )

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# clf.fit(X_train, y_train)
# print("model score: %.3f" % clf.score(X_test, y_test))


In [28]:
from sklearn import svm

# Fit the model(took about 2 mins)
clf = svm.SVC(kernel="linear", C=1000) # this C parameter can actually be very wrong, but ok for now
clf.fit(X, y_hat)

SVC(C=1000, kernel='linear')

In [40]:
y_test = df_test.pop("class")
X_test = df_test

y_test_hat = le.transform(y_test)

### Check the performance on the rest of the dataframe

The low value here can be because of the absence of any kind of data prep or because of the concept drift inherent to the dataset.

In [43]:
clf.score(X_test, y_test_hat)

0.46325075418228356

### Check the performance on the training dataset just for curiosity

It's a little bit low. 

In [44]:
clf.score(X, y_hat)

0.6568422519509476

## Check for some better performance when using a grid searched classifier

In [50]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[1, 10, 100, 1000]}

svc = svm.SVC()
grid_clf = GridSearchCV(svc, parameters) # took 30mins
grid_clf.fit(X, y_hat)

sorted(grid_clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [52]:
grid_clf.score(X_test, y_test_hat)

0.47313892190023465

## TODO: check the feature importance

## TODO: Plot decision boundaries for pairs of attribures

In [39]:
# import matplotlib.pyplot as plt
# from sklearn.inspection import DecisionBoundaryDisplay

# plt.scatter(X["Att1"], X["Att2"], c=y_hat, s=30, cmap=plt.cm.Paired)

# # plot the decision function
# ax = plt.gca()

# DecisionBoundaryDisplay.from_estimator(
#     clf,
#     X,
#     plot_method="contour",
#     colors="k",
#     levels=[-1, 0, 1],
#     alpha=0.5,
#     linestyles=["--", "-", "--"],
#     ax=ax,
# )

# # plot support vectors
# ax.scatter(
#     clf.support_vectors_[:, "Att1"],
#     clf.support_vectors_[:, "Att2"],
#     s=100,
#     linewidth=1,
#     facecolors="none",
#     edgecolors="k",
# )

# plt.show()